
pip intall openpyxl

import pandas as pd
import numpy as np
import os
from IPython.display import clear_output

In [15]:
import pandas as pd
import numpy as np
import os


# Tipos do espd0068
espd068Type = {
    'Repres': 'int64',                 
    'Nome Repres': 'object',          
    'Estab': 'int64',                  
    'Nr Nota Fiscal': 'int64',         
    'Dt Emis Nota': 'str',          
    'Cod Cliente': 'int64',           
    'Nome Cliente': 'object',          
    'Pedido': 'int64',                 
    'Cod Item': 'int64',               
    'Desc Item': 'object',            
    'Qtde Faturada': 'int64',         
    'Valor Unit Item': 'object',       
    'Valor Total Item': 'object',      
    'Qtd Nao Atendida': 'int64',       
    'Valor Nao Atendido': 'object'
}

# Tipos do espd0028
espd028Type = {
    'Data_Implant': 'str',
    'Nr_Pedido': 'int64',
    'Nr_Cliente': 'int64',
    'Nome_Abrev': 'object',
    'Nome_Repre': 'object',
    'Familia': 'object', # Este campo está em um lugar incorreto no seu arquivo
    'Codigo_Item': 'int64',
    'Descricao': 'object',
    'Qt_Pedido': 'object',
    'R_Unit': 'object',
    'Qt_Aloc': 'int64',
    'Vlr_Total': 'object',
    'Est_Disp': 'object',
    'Situacao': 'object',
    'Entrega': 'str',
    'MesAno': 'int64',
    'Lista_Presente': 'int64',
    'Observacao': 'object',
    # As colunas abaixo parecem ser lixo e precisam ser ignoradas
    'Lixo1': 'object',
    'Lixo2': 'object',
    'Lixo3': 'object',
    'Lixo4': 'object',
    'Lixo5': 'object',
    'Lixo6': 'object',
    'Lixo7': 'object',
}

In [16]:
import pandas as pd
import os

# Lista dos arquivos que você quer empilhar
arquivos_espd028 = [
    'espd0028 - LEGO.txt',
    'espd0028 - MGA.txt',
    'espd0028 - POK.txt'
]

lista_dfs = []

# Nomes das colunas conforme seu código original
nomes_colunas = [
    'Data_Implant', 'Nr_Pedido', 'Nr_Cliente', 'Nome_Abrev', 'Nome_Repre',
    'Familia', 'Codigo_Item', 'Descricao', 'Qt_Pedido', 'R_Unit',
    'Qt_Aloc', 'Vlr_Total', 'Est_Disp', 'Situacao', 'Entrega',
    'MesAno', 'Lista_Presente', 'Observacao',
    'Coluna19', 'Coluna20', 'Coluna21', 'Coluna22', 'Coluna23', 'Coluna24', 'Coluna25'
]

# Loop para ler e empilhar
for arquivo in arquivos_espd028:
    if os.path.exists(arquivo):
        df_temp = pd.read_csv(
            arquivo,
            sep='@',
            encoding='ISO-8859-1',
            header=None,
            names=nomes_colunas,
            skiprows=1,
            dtype=espd028Type
        )
        lista_dfs.append(df_temp)

# Aqui a variável espd028 passa a conter os 3 arquivos juntos
espd028 = pd.concat(lista_dfs, ignore_index=True)

# Remove as colunas inúteis (mantendo sua lógica original)
colunas_para_remover = [
    'Coluna19', 'Coluna20', 'Coluna21', 'Coluna22', 'Coluna23', 'Coluna24', 'Coluna25'
]
espd028 = espd028.drop(columns=colunas_para_remover)


In [17]:
espd068_path = 'espd0068.txt'

espd068 = pd.read_csv(espd068_path, sep='@', quotechar='"', encoding='ISO-8859-1',dtype=espd068Type)


espd068['Dt Emis Nota'] = pd.to_datetime(espd068['Dt Emis Nota'], format='%d/%m/%y')
espd028['Data_Implant'] = pd.to_datetime(espd028['Data_Implant'], format='%d/%m/%y')
espd028 = espd028.dropna(axis=1, how='all')

espd028['Qt_Pedido'] = espd028['Qt_Pedido'].str.replace(',', '.', regex=False)

espd028['Est_Disp'] = espd028['Est_Disp'].str.replace(',', '.', regex=False).astype(float).round().astype('Int64')



In [18]:
pedidos_validados =  [3627243,3627318,3638006]
mascara_para_manter = ~espd028['Nr_Pedido'].isin(pedidos_validados)
espd028 = espd028[mascara_para_manter]

In [19]:

itens_codigo_especifico = espd028[espd028['Codigo_Item'] == 4111160404]

itens_codigo_especifico = itens_codigo_especifico[['Codigo_Item', 'Est_Disp']]



####### Consolidando os dados no espd068 para cada combinação de Pedido e Cod Item

In [20]:


##espd028.to_excel('espd0028 base.xlsx',index=False)
espd068_consolidado = espd068.groupby(['Pedido', 'Cod Item'], as_index=False)['Qtde Faturada'].sum()


espd028 = espd028.merge(
    espd068_consolidado,
    left_on=['Nr_Pedido', 'Codigo_Item'],
    right_on=['Pedido', 'Cod Item'],
    how='left'
)


espd028 = espd028.rename(columns={'Qtde Faturada': 'Qtde_Faturada'})


espd028 = espd028.drop(columns=['Pedido', 'Cod Item'])


espd028['Qtde_Faturada'] = espd028['Qtde_Faturada'].fillna(0).astype('Int64')



In [21]:
qtde_faturada_maior_que_zero = espd028[espd028['Qtde_Faturada'] > 0]


In [22]:

espd028['Qt_Aloc'] = espd028['Qt_Aloc'] - espd028['Qtde_Faturada']

In [23]:
qtde_faturada_maior_que_zero = espd028[espd028['Qtde_Faturada'] > 0]

In [24]:
import numpy as np


espd028['Qt_Pedido'] = np.where(
    espd028['Situacao'] == 'Total', 
    0, 
    np.where(
        espd028['Qt_Pedido'].fillna(0).astype(float) - espd028['Qt_Aloc'].fillna(0).astype(float) < 0, 
        0, 
        espd028['Qt_Pedido'].fillna(0).astype(float) - espd028['Qt_Aloc'].fillna(0).astype(float)
    )
)

espd028['Qt_Pedido'] = espd028['Qt_Pedido'].astype(float).round().astype('Int64')



In [25]:
transito = pd.read_excel('transito.xlsx', header=0)

text_columns = transito.columns[transito.columns.map(lambda x: isinstance(x, str))]

espd028 = espd028.merge(
    transito,
    left_on='Codigo_Item',
    right_on='Cod',
    how='left'
)
for column in text_columns:
    espd028[column] = espd028[column].fillna(0).astype(int)
espd028 = espd028.drop(columns=['Cod'])

In [26]:
text_columns = transito.columns.tolist()
if 'Cod' in text_columns:
    text_columns.remove('Cod')
text_columns.insert(0, 'Est_Disp')

In [27]:
estoque_inicial = espd028[['Codigo_Item'] + text_columns].copy()


estoque_inicial = estoque_inicial.drop_duplicates(subset='Codigo_Item')


estoque_inicial[text_columns] = estoque_inicial[text_columns].fillna(0).astype(int)


In [ ]:
import random


espd028.to_excel('espd0028 final.xlsx',index=False)

espd028['Valor Item'] = espd028['R_Unit'].apply(lambda x: float(x.replace(',', '.'))) * espd028['Qt_Pedido']


pedidos_totais = espd028.groupby('Nr_Pedido', as_index=False)['Valor Item'].sum()
pedidos_totais = pedidos_totais.rename(columns={'Valor Item': 'Valor Total Pedido'})


espd028 = espd028.merge(pedidos_totais, on='Nr_Pedido', how='left')


pedidos = espd028['Nr_Pedido'].unique()


pedidos_maior_para_menor = list(pedidos_totais.sort_values(by='Valor Total Pedido', ascending=False)['Nr_Pedido'])

pedidos_menor_para_maior = list(pedidos_totais.sort_values(by='Valor Total Pedido', ascending=True)['Nr_Pedido'])

pedidos_aleatorio1 = list(pedidos)
pedidos_aleatorio2 = list(pedidos)
random.shuffle(pedidos_aleatorio1)
random.shuffle(pedidos_aleatorio2)

pedidos_recente_para_antigo = list(
    espd028.sort_values(by='Data_Implant', ascending=False)
    .drop_duplicates(subset='Nr_Pedido')['Nr_Pedido']
)

# Opcional: Do mais antigo para o mais recente
pedidos_antigo_para_recente = list(
    espd028.sort_values(by='Data_Implant', ascending=True)
    .drop_duplicates(subset='Nr_Pedido')['Nr_Pedido']
)

string_pedidos = "3586806;3592341;3616538;3635988;3654450;3672942;3727084;3736323;3772600;3566181;3598573;3598571;3598650;3603908;3604874;3612416;3616553;3623818;3626923;3626921;3643231;3645344;3654485;3661202;3674278;3691483;3691494;3713558;3719089;3719090;3719409;3723128;3740534;3745212;3745211;3745076;3744483;3748663;3759655;3759477;3759480"
pedidos_prioritarios = [int(p) for p in string_pedidos.split(';') if p.strip()]

todos_pedidos = list(pedidos)

pedidos_na_lista = [p for p in pedidos_prioritarios if p in todos_pedidos]
pedidos_fora_da_lista = [p for p in todos_pedidos if p not in pedidos_prioritarios]

pedidos_customizados = pedidos_na_lista + pedidos_fora_da_lista

estrategias = {
    'Prioridade Manual': pedidos_customizados,
    'Maior para Menor': pedidos_maior_para_menor,
    'Menor para Maior': pedidos_menor_para_maior,
    'Aleatório 1': pedidos_aleatorio1,
    'Aleatório 2': pedidos_aleatorio2,
    'recente para antigo':pedidos_recente_para_antigo,
    'antigo para recente':pedidos_antigo_para_recente
}

def simular_faturamento(pedidos_ordem, espd028, estoque_inicial, text_columns):
    faturado_total = []

    estoque_sim = estoque_inicial.copy(deep=True)
    ordem_faturamento = 0

    for pedido in pedidos_ordem:
        ordem_faturamento += 1  
        itens_pedido = espd028[espd028['Nr_Pedido'] == pedido].copy()

       
        for estoque_col in text_columns:
         
            for idx, item in itens_pedido.iterrows():
                qt_pedido = item['Qt_Pedido']
                if qt_pedido <= 0:
                    continue 

                cliente = item['Nr_Cliente']
                familia = item['Familia']
                codigo_item = item['Codigo_Item']
                valor_unit = float(item['R_Unit'].replace(',', '.'))

               
                if codigo_item not in estoque_sim['Codigo_Item'].values:
                    continue

              
                estoque_item = estoque_sim[estoque_sim['Codigo_Item'] == codigo_item].iloc[0]
                estoque_atual = estoque_item[estoque_col]

                if estoque_atual > 0:
                    qt_faturada = min(qt_pedido, estoque_atual)
                    valor_faturado = qt_faturada * valor_unit

                    faturado_total.append({
                        'Nr_Pedido': pedido,
                        'Nr_Cliente': cliente,
                        'Familia': familia,
                        'Codigo_Item': codigo_item,
                        'Qtde Faturada': qt_faturada,
                        'Valor Faturado': valor_faturado,
                        'Estoque Faturado': estoque_col,
                        'Ordem Faturamento Pedido': ordem_faturamento,
                    })

                  
                    itens_pedido.loc[idx, 'Qt_Pedido'] -= qt_faturada
                    estoque_sim.loc[estoque_sim['Codigo_Item'] == codigo_item, estoque_col] -= qt_faturada

    
    return pd.DataFrame(faturado_total)

with pd.ExcelWriter('Faturado_Total_Estrategias.xlsx') as writer:
    for nome_estrategia, pedidos_ordem in estrategias.items():
        resultado = simular_faturamento(pedidos_ordem, espd028, estoque_inicial, text_columns)
        resultado.to_excel(writer, sheet_name=nome_estrategia, index=False)

print('Simulação de faturamento por pedido concluída. Arquivo "Faturado_Total_Estrategias.xlsx" gerado.')